In [1]:
import pip

package_names = ['lightgbm', 'awswrangler', 'sagemaker'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for sagemaker: filename=sagemaker-2.49.2-py2.py3-none-any.whl size=595773 sha256=2944d9ba89197b6fad432b35891bfff0a19d049495591ad1573aa64d2fc6e126
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9e/d2/fe/3693f819a8e9d264ec928096add080eef4112383aa8b92b698
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.45.0
    Uninstalling sagemaker-2.45.0:
      Successfully uninstalled sagemaker-2.45.0


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

Generating new fontManager, this may take some time...
NumExpr defaulting to 4 threads.


### CONFIG

In [3]:
proyecto = 'propension'
#tabla_universo = 'T_TEST_UNIVERSO_202102'
tabla_universo = 'HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE'

now = datetime.now()
sess = sagemaker.session.Session()
s3 = boto3.resource('s3')
bucket = sess.default_bucket() 
region = boto3.Session().region_name

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-07-28 07:45:39.469266


In [4]:
uri_universo_athena = 's3://{}/vpc/{}/athena/{}'.format(bucket, proyecto, tabla_universo)
uri_output = 's3://{}/vpc/{}/output'.format(bucket, proyecto)
uri_code = 's3://{}/vpc/{}/code'.format(bucket, proyecto)

print('uri_universo_athena: ', uri_universo_athena)
print('uri_output: ', uri_output)
print('uri_code: ', uri_code)

print("")
print("/"*50, "\n")
path_container_input = '/opt/ml/processing/input'
path_container_output = path_container_input.replace('input', 'output')
path_container_utils= path_container_input + '/utils'
path_container_universo = path_container_input + '/' + tabla_universo

print('path_container_input: ', path_container_input)
print('path_container_output: ', path_container_output)
print('path_container_utils: ', path_container_utils)
print('path_container_universo: ', path_container_universo)

uri_universo_athena:  s3://sagemaker-us-east-1-058528764918/vpc/propension/athena/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
uri_output:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output
uri_code:  s3://sagemaker-us-east-1-058528764918/vpc/propension/code

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE


### SETEADO DE UTILITARIO

In [5]:
BASE_DIR

'/home/ec2-user/SageMaker'

In [6]:
path_instancia_util = os.path.join(BASE_DIR, 'PROPENSION/UTILITARIO_CODE/utils.py')
path_instancia_util

'/home/ec2-user/SageMaker/PROPENSION/UTILITARIO_CODE/utils.py'

In [7]:
prefix_utils = '{}/utils.py'.format(uri_code.split(bucket)[-1][1:])
prefix_utils

'vpc/propension/code/utils.py'

In [8]:
s3.Bucket(bucket).upload_file(path_instancia_util, prefix_utils)

### DESACOPLADO DIRECTO S3

In [9]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor_a = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.12xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [10]:
%%time
processing_job_name = "propension-workflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

sklearn_processor_a.run(
    code='feature_partition.py',  #'/opt/ml/processing/codigo_proceso/' , '/opt/ml/processing/input/code'
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=uri_universo_athena,
            destination=path_container_universo,
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source=uri_code,
            destination=path_container_utils,
            s3_data_distribution_type='ShardedByS3Key'
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=uri_output,
            source=path_container_output
        )
    ]
)

preprocessing_job_description = sklearn_processor_a.jobs[-1].describe()


Job Name:  propension-workflow-2021-07-28-07-45-40
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/athena/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE', 'LocalPath': '/opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/code', 'LocalPath': '/opt/ml/processing/input/utils', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/propension-workflow-2021-07-28-07-45-40/input/code/feature_partition.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'Fil

NameError: name 'sklearn_processor1' is not defined

In [11]:
#    arguments=["--ultimo_mes", ultimo_mes]